In [2]:
! python -m pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 91.7 MB/s eta 0:00:00


In [ ]:
import json
import gurobipy as gp
from gurobipy import GRB
import pandas as pd


# LETTURA DEI DATI
df = pd.read_csv("telecomandi.csv")
# imposto la colonna i come indice di riga per poter accedere ad una riga del file utilizzando .loc
df = df.set_index('i')

print(df.head)


# seleziono colonna indicizzata da "quantita"
telecomando_A = df["telecomando_A"].to_dict()
print(telecomando_A)

# DEFINIZIONE DEGLI INSIEMI
## Parametri P: profitto unitario per il bene i in I
"""
seleziono la riga con indice 'profitto' e la converto in dizionario le cui chiavi saranno i valori delle colonne
P = { 'telecomando_A' : 3.0,
      'telecomando_B' : 8.0 }
"""
P = df.loc["profitto"].to_dict()

## Parametri Q: quantità disponibile della risorsa j in J
"""
seleziono la colonna 'quantita' e la converto in dizionario le cui chiavi saranno gli indici delle righe

Q = { "display": 10,
      "logica": 18,
      "trasmettitori": 12,
      "tastierini": 21,
      "navigazione": 9,
      "led": 10 }
"""
Q = df["quantita"].to_dict()

## Parametri A (matrice): quantità di risorsa j necessaria per la produzione di un'unità di del bene i
"""
rimuovo la riga 'profitto' e la colonna 'quantita' per ottenere la matrice A
A[j,i] è la quantita di risorsa j necessaria per la produzione di un'unità di bene i
NB: gli indici sono A_ji e non A_ij per come è strutturato il file csv
"""
A  = df.drop(index="profitto").drop(columns="quantita")

## Insieme I dei beni  che possono essere prodotti
"""
seleziono i nomi delle colonne e li converto in lista
I = ['telecomando_A', 'telecomando_B']
"""
I = A.columns.tolist()

## Insieme J delle risorse disponibli
"""
seleziono gli indici delle righe e li converto in lista
J = ['display', 'logica', 'trasmettitori', 'tastierini', 'navigazione', 'led']
"""
J = A.index.tolist()



print("Beni (I):", I)
print("Risorse (J):", J)
print("Profitti (P):", P)
print("Quantità (D):", Q)
print("Constraint Matrix (A):\n", A)


# Creazione modello
m = gp.Model("mix_opt_telecomandi")
m.setParam("OutputFlag", 0)

# Creazione variabili decisionali
x = {}
for i in I:
  x[f"{i}"] = m.addVar(vtype=GRB.INTEGER, lb=0, name=f"x_{i}")
  # x[i] è equivalente

# equivalente anche la seguente riga di codice
# x = m.addVars(I, vtype=GRB.INTEGER, lb=0, name="x")

# Aggiunta vincoli
for j in J:
    sum = gp.quicksum(A.loc[j, i] * x[i] for i in I)
    m.addConstr(
        sum <= Q[j],
        name=f"vincolo_{j}",
    )

# O.f.
m.setObjective(gp.quicksum(P[i] * x[i] for i in I), GRB.MAXIMIZE)

m.optimize()

# Visualizzazione risultato
status_names = {
    GRB.OPTIMAL: "OPTIMAL",
    GRB.SUBOPTIMAL: "SUBOPTIMAL",
    GRB.INFEASIBLE: "INFEASIBLE",
    GRB.UNBOUNDED: "UNBOUNDED",
    GRB.INF_OR_UNBD: "INF_OR_UNBD",
}
print(f"\nModel status: {status_names.get(m.status, m.status)}")

if m.status == GRB.OPTIMAL:
    print(f"Objective (costo) = {m.objVal:.6g}\n")
    for i in I:
        if x[i].X > 1e-6:
            print(f"x[{i}] = {x[i].X:.0f}")

<bound method NDFrame.head of                telecomando_A  telecomando_B  quantita
i                                                    
display                    1              2      10.0
logica                     2              2      18.0
trasmettitori              1              1      12.0
tastierini                 2              3      21.0
navigazione                1              0       9.0
led                        1              1      10.0
profitto                   3              8       NaN>
{'display': 1, 'logica': 2, 'trasmettitori': 1, 'tastierini': 2, 'navigazione': 1, 'led': 1, 'profitto': 3}
valore: 1
Beni (I): ['telecomando_A', 'telecomando_B']
Risorse (J): ['display', 'logica', 'trasmettitori', 'tastierini', 'navigazione', 'led']
Profitti (P): {'telecomando_A': 3.0, 'telecomando_B': 8.0, 'quantita': nan}
Quantità (D): {'display': 10.0, 'logica': 18.0, 'trasmettitori': 12.0, 'tastierini': 21.0, 'navigazione': 9.0, 'led': 10.0, 'profitto': nan}
Constraint Matr